In [ ]:
# Setup and Initialization
from dash import Dash, dcc, html, dash_table
import dash_leaflet as dl
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from CRUD import AnimalShelter
import base64
from dash.exceptions import PreventUpdate
import dash
import json

print("Setting up the dashboard...")

# Initialize database connection
shelter = AnimalShelter()

# Load initial data with pagination
initial_data = shelter.read({}, page=1, page_size=10)
df = pd.DataFrame(initial_data['data'])

# Clean data by removing MongoDB ObjectId
if '_id' in df.columns:
    df = df.drop('_id', axis=1)

# Load and encode the company logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('utf-8')

# Initialize Dash application
app = Dash('GraziosoSalvare')

# Define visualization color scheme
colors = {
    'breed_chart': 'rgb(41, 128, 185)',    # Blue
    'age_chart': 'rgb(46, 204, 113)',      # Green
    'outcome_chart': {
        'Return to Owner': 'rgb(52, 152, 219)',  # Blue
        'Adoption': 'rgb(46, 204, 113)',         # Green
        'Transfer': 'rgb(241, 196, 15)',         # Yellow
        'Euthanasia': 'rgb(231, 76, 60)',        # Red
        'Died': 'rgb(149, 165, 166)'             # Gray
    }
}

# Define rescue type criteria
rescue_criteria = {
    "Water Rescue": {
        "breeds": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": "Intact Female",
        "age_min": 26,
        "age_max": 156
    },
    "Mountain Rescue": {
        "breeds": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                  "Siberian Husky", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 26,
        "age_max": 156
    },
    "Disaster Rescue": {
        "breeds": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                  "Bloodhound", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 20,
        "age_max": 300
    }
}

# Dashboard Layout
app.layout = html.Div([
    # Header Section with Logo and Title
    html.Div([
        html.A([
            html.Center(html.Img(
                src=f'data:image/png;base64,{encoded_image}',
                style={
                    'height': '200px',
                    'width': '200px',
                    'marginTop': '20px'
                }
            ))
        ], href='https://www.snhu.edu', target='_blank'),
        html.Center(html.H1('Grazioso Salvare Animal Dashboard', 
            style={
                'fontSize': '36px',
                'fontWeight': 'bold',
                'color': '#2c3e50',
                'marginTop': '20px',
                'fontFamily': 'Arial, sans-serif'
            }
        )),
        html.Center(html.H3('Find and track rescue animals',
            style={
                'fontSize': '18px',
                'color': '#7f8c8d',
                'marginBottom': '30px',
                'fontFamily': 'Arial, sans-serif'
            }
        ))
    ], style={
        'backgroundColor': 'white',
        'padding': '20px 0',
        'borderRadius': '8px',
        'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
        'marginBottom': '30px'
    }),

    # Controls Container - Filters and Pagination
    html.Div([
        # Filter Controls
        html.Div([
            # Rescue Type Dropdown
            html.Div([
                html.Label('Rescue Type:', 
                    style={
                        'fontWeight': 'bold',
                        'marginBottom': '8px',
                        'color': '#2c3e50'
                    }
                ),
                dcc.Dropdown(
                    id='rescue-type-filter',
                    options=[{'label': k, 'value': k} for k in rescue_criteria.keys()] + [{'label': 'All', 'value': 'All'}],
                    value='All',
                    clearable=False,
                    style={
                        'borderRadius': '4px',
                        'border': '1px solid #dcdcdc'
                    }
                )
            ], style={
                'width': '30%', 
                'display': 'inline-block', 
                'marginRight': '20px',
                'backgroundColor': 'white',
                'padding': '15px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.05)'
            }),
            
            # Age Range Slider
            html.Div([
                html.Label('Age Range (years):', 
                    style={
                        'fontWeight': 'bold',
                        'marginBottom': '8px',
                        'color': '#2c3e50'
                    }
                ),
                dcc.RangeSlider(
                    id='age-range-slider',
                    min=0,
                    max=10,
                    step=0.5,
                    marks={
                        i: {'label': str(i), 'style': {'color': '#77778c'}}
                        for i in range(11)
                    },
                    value=[0, 10],
                    tooltip={
                        "placement": "bottom",
                        "always_visible": True
                    },
                    allowCross=False,
                    className="custom-slider"
                ),
                html.Div(
                    id='slider-output-container',
                    style={
                        'textAlign': 'center',
                        'marginTop': '10px',
                        'color': '#666',
                        'fontSize': '14px'
                    }
                )
            ], style={
                'width': '65%',
                'display': 'inline-block',
                'backgroundColor': 'white',
                'padding': '15px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.05)'
            })
        ], style={
            'marginBottom': '20px',
            'display': 'flex',
            'justifyContent': 'space-between',
            'alignItems': 'flex-start'
        }),

        # Pagination Controls
        html.Div([
            # Previous Page Button
            html.Button('Previous', 
                id='prev-page-button', 
                n_clicks=0,
                style={
                    'backgroundColor': '#3498db',
                    'color': 'white',
                    'border': 'none',
                    'padding': '10px 20px',
                    'borderRadius': '4px',
                    'cursor': 'pointer',
                    'marginRight': '10px',
                    'fontWeight': 'bold'
                }
            ),
            # Page Number Input
            html.Div([
                html.Span('Page: ', style={'marginRight': '5px', 'color': '#2c3e50'}),
                dcc.Input(
                    id='page-input',
                    type='number',
                    min=1,
                    step=1,
                    value=1,
                    style={
                        'width': '60px',
                        'marginRight': '5px',
                        'padding': '5px',
                        'borderRadius': '4px',
                        'border': '1px solid #dcdcdc',
                        'textAlign': 'center'
                    }
                ),
                html.Span(id='page-display', style={'marginRight': '10px', 'color': '#2c3e50'})
            ], style={'display': 'inline-block', 'margin': '0 10px'}),
            # Next Page Button
            html.Button('Next', 
                id='next-page-button', 
                n_clicks=0,
                style={
                    'backgroundColor': '#3498db',
                    'color': 'white',
                    'border': 'none',
                    'padding': '10px 20px',
                    'borderRadius': '4px',
                    'cursor': 'pointer',
                    'marginRight': '20px',
                    'fontWeight': 'bold'
                }
            ),
            # Items Per Page Selector
            html.Div([
                html.Label('Items per page: ', style={'color': '#2c3e50', 'marginRight': '10px'}),
                dcc.Dropdown(
                    id='page-size-dropdown',
                    options=[
                        {'label': '10', 'value': 10},
                        {'label': '20', 'value': 20},
                        {'label': '50', 'value': 50}
                    ],
                    value=10,
                    clearable=False,
                    style={
                        'width': '100px',
                        'display': 'inline-block'
                    }
                )
            ], style={
                'display': 'inline-flex',
                'alignItems': 'center'
            })
        ], style={
            'backgroundColor': 'white',
            'padding': '15px',
            'borderRadius': '8px',
            'boxShadow': '0 2px 4px rgba(0,0,0,0.05)',
            'textAlign': 'center',
            'marginBottom': '20px'
        })
    ], style={
        'backgroundColor': '#f8f9fa',
        'padding': '20px',
        'borderRadius': '8px',
        'marginBottom': '20px'
    }),

    # Data Table Component
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": "ID", "id": "animal_id"},
            {"name": "Name", "id": "name"},
            {"name": "Type", "id": "animal_type"},
            {"name": "Breed", "id": "breed"},
            {"name": "Color", "id": "color"},
            {"name": "Age", "id": "age_upon_outcome"},
            {"name": "Sex", "id": "sex_upon_outcome"},
            {"name": "Outcome", "id": "outcome_type"},
            {"name": "Details", "id": "outcome_subtype"},
            {"name": "Birth Date", "id": "date_of_birth"},
            {"name": "Record Date", "id": "datetime"}
        ],
        data=df.to_dict('records'),
        page_action='none',
        page_size=10,
        style_table={
            'overflowX': 'auto',
            'borderRadius': '8px',
            'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
        },
        # Base cell styling
        style_cell={
            'textAlign': 'left',
            'padding': '12px 15px',
            'backgroundColor': 'white',
            'fontSize': '14px',
            'fontFamily': 'Arial, sans-serif',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },
        # Column-specific styling
        style_cell_conditional=[
            {'if': {'column_id': 'select'}, 
             'width': '75px',
             'minWidth': '75px',
             'padding': '12px 20px'},
            {'if': {'column_id': 'animal_id'}, 'width': '100px', 'minWidth': '100px', 'textAlign': 'center'},
            {'if': {'column_id': 'name'}, 'width': '150px', 'minWidth': '150px'},
            {'if': {'column_id': 'animal_type'}, 'width': '100px', 'minWidth': '100px'},
            {'if': {'column_id': 'breed'}, 'width': '200px', 'minWidth': '200px'},
            {'if': {'column_id': 'color'}, 'width': '150px', 'minWidth': '150px'},
            {'if': {'column_id': 'age_upon_outcome'}, 'width': '100px', 'minWidth': '100px'},
            {'if': {'column_id': 'sex_upon_outcome'}, 'width': '150px', 'minWidth': '150px'},
            {'if': {'column_id': 'outcome_type'}, 'width': '120px', 'minWidth': '120px'},
            {'if': {'column_id': 'outcome_subtype'}, 'width': '150px', 'minWidth': '150px'},
            {'if': {'column_id': 'date_of_birth'}, 'width': '120px', 'minWidth': '120px', 'textAlign': 'center'},
            {'if': {'column_id': 'datetime'}, 'width': '120px', 'minWidth': '120px', 'textAlign': 'center'}
        ],
        # Header styling
        style_header={
            'backgroundColor': '#f8f9fa',
            'fontWeight': 'bold',
            'borderBottom': '2px solid #dee2e6',
            'textAlign': 'center',
            'whiteSpace': 'normal',
            'height': 'auto',
            'padding': '12px 15px'
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
        },
        # Row selection settings
        row_selectable='multi',
        selected_rows=[],
        # Conditional row styling
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': '#f8f9fa'
            },
            {
                'if': {'state': 'selected'},
                'backgroundColor': '#e6f3ff',
                'border': '1px solid #2196F3'
            }
        ],
        # Sorting configuration
        sort_action='custom',
        sort_mode='single',
        # Column tooltips
        tooltip_header={
            'animal_id': 'Unique Identifier',
            'name': 'Animal Name',
            'animal_type': 'Type of Animal',
            'breed': 'Animal Breed',
            'color': 'Animal Color',
            'age_upon_outcome': 'Age at Time of Record',
            'sex_upon_outcome': 'Sex and Spay/Neuter Status',
            'outcome_type': 'Type of Outcome',
            'outcome_subtype': 'Additional Outcome Details',
            'date_of_birth': 'Date of Birth',
            'datetime': 'Record Date'
        },
        tooltip_delay=0,
        tooltip_duration=None
    ),
    
        html.Hr(),

    # Map Container Component
    html.Div(
        id='map-container',
        children=[
            dl.Map(
                id='animal-map',
                style={'width': '100%', 'height': '500px'},
                center=[30.75, -97.48],
                zoom=9,
                children=[
                    # OpenStreetMap tile layer
                    dl.TileLayer(url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png"),
                    # Dynamic marker layer group
                    dl.LayerGroup(id="marker-group")
                ],
                scrollWheelZoom=False,
                doubleClickZoom=True
            )
        ]
    ),

    html.Hr(),

    # Statistics Section
    html.Div([
        # Statistics Cards Container
        html.Div([
            # Total Animals Card
            html.Div([
                html.H4('Total Animals', style={
                    'color': '#7f8c8d',
                    'fontSize': '16px',
                    'marginBottom': '10px'
                }),
                html.H2(id='total-animals-count', style={
                    'color': '#2c3e50',
                    'fontSize': '28px',
                    'margin': '0'
                })
            ], style={
                'backgroundColor': 'white',
                'padding': '20px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
                'flex': '1',
                'textAlign': 'center',
                'minWidth': '180px'
            }),

            # Average Age Card
            html.Div([
                html.H4('Average Age', style={
                    'color': '#7f8c8d',
                    'fontSize': '16px',
                    'marginBottom': '10px'
                }),
                html.H2(id='average-age', style={
                    'color': '#2c3e50',
                    'fontSize': '28px',
                    'margin': '0'
                })
            ], style={
                'backgroundColor': 'white',
                'padding': '20px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
                'flex': '1',
                'textAlign': 'center',
                'minWidth': '180px'
            }),

            # Unique Breeds Card
            html.Div([
                html.H4('Unique Breeds', style={
                    'color': '#7f8c8d',
                    'fontSize': '16px',
                    'marginBottom': '10px'
                }),
                html.H2(id='breed-count', style={
                    'color': '#2c3e50',
                    'fontSize': '28px',
                    'margin': '0'
                })
            ], style={
                'backgroundColor': 'white',
                'padding': '20px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
                'flex': '1',
                'textAlign': 'center',
                'minWidth': '180px'
            }),

            # Most Common Sex Card
            html.Div([
                html.H4('Most Common Sex', style={
                    'color': '#7f8c8d',
                    'fontSize': '16px',
                    'marginBottom': '10px'
                }),
                html.H2(id='common-sex', style={
                    'color': '#2c3e50',
                    'fontSize': '28px',
                    'margin': '0'
                })
            ], style={
                'backgroundColor': 'white',
                'padding': '20px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
                'flex': '1',
                'textAlign': 'center',
                'minWidth': '180px'
            }),

            # Positive Outcome Rate Card
            html.Div([
                html.H4('Positive Outcome Rate', style={
                    'color': '#7f8c8d',
                    'fontSize': '16px',
                    'marginBottom': '10px'
                }),
                html.H2(id='success-rate', style={
                    'color': '#2c3e50',
                    'fontSize': '28px',
                    'margin': '0'
                }),
                html.Div(
                    "Percentage of animals returned to owners or adopted",
                    style={
                        'fontSize': '12px',
                        'color': '#95a5a6',
                        'marginTop': '5px'
                    }
                )
            ], style={
                'backgroundColor': 'white',
                'padding': '20px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
                'flex': '1',
                'textAlign': 'center',
                'minWidth': '180px'
            })
        ], style={
            'display': 'flex',
            'gap': '20px',
            'marginBottom': '30px',
            'flexWrap': 'wrap'
        }),

        # Charts Container
        html.Div([
            # Top Row Charts
            html.Div([
                # Breed Distribution Chart
                html.Div([
                    dcc.Graph(id='breed-chart')
                ], style={
                    'width': '50%',
                    'backgroundColor': 'white',
                    'padding': '20px',
                    'borderRadius': '8px',
                    'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'
                }),
                
                # Age Distribution Chart
                html.Div([
                    dcc.Graph(id='age-chart')
                ], style={
                    'width': '50%',
                    'backgroundColor': 'white',
                    'padding': '20px',
                    'borderRadius': '8px',
                    'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'
                })
            ], style={
                'display': 'flex',
                'gap': '20px',
                'marginBottom': '20px'
            }),

            # Bottom Row Chart
            html.Div([
                dcc.Graph(id='outcome-chart')
            ], style={
                'backgroundColor': 'white',
                'padding': '20px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'
            })
        ])
    ], style={
        'backgroundColor': '#f8f9fa',
        'padding': '20px',
        'borderRadius': '8px',
        'marginTop': '30px'
    }),

    # State Management Components
    dcc.Store(id='page-data'),
    dcc.Store(id='filter-data')
])

# Callback for Map Updates
@app.callback(
    [Output('marker-group', 'children'),
     Output('animal-map', 'center'),
     Output('animal-map', 'zoom')],
    [Input('datatable-id', 'data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(data, selected_rows):
    """
    Updates map markers and view based on data table selection.
    
    Args:
        data: Current data in the table
        selected_rows: Indices of selected rows in the table
        
    Returns:
        markers: List of map markers
        center: Center coordinates for map view
        fixed_zoom: Constant zoom level
    """
    df = pd.DataFrame(data)
    markers = []
    default_center = [30.75, -97.48]
    fixed_zoom = 9  # Constant zoom level

    if not df.empty:
        # Filter data based on selection
        if selected_rows:
            display_df = df.iloc[selected_rows]
        else:
            display_df = df
            
        # Get valid coordinates
        valid_coords = display_df[display_df['location_lat'].notna() & display_df['location_long'].notna()]
        
        if not valid_coords.empty:
            # Create markers for each location
            for _, row in valid_coords.iterrows():
                markers.append(
                    dl.Marker(
                        position=[float(row['location_lat']), float(row['location_long'])],
                        children=[
                            dl.Tooltip(f"{row['name'] if pd.notna(row['name']) and row['name'] != '' else 'Unnamed'} (ID: {row['animal_id']})"),
                            dl.Popup([
                                html.H4(row['name'] if pd.notna(row['name']) and row['name'] != '' else 'Unnamed'),
                                html.P(f"ID: {row['animal_id']}"),
                                html.P(f"Breed: {row['breed']}"),
                                html.P(f"Age: {row['age_upon_outcome']}"),
                                html.P(f"Sex: {row['sex_upon_outcome']}")
                            ])
                        ]
                    )
                )
            
            # Calculate center point for map view
            center = [
                float(valid_coords['location_lat'].mean()),
                float(valid_coords['location_long'].mean())
            ]
            return markers, center, fixed_zoom
    
    return markers, default_center, fixed_zoom

# Callback for Statistics and Charts Updates
@app.callback(
    [Output('total-animals-count', 'children'),
     Output('average-age', 'children'),
     Output('breed-count', 'children'),
     Output('common-sex', 'children'),
     Output('success-rate', 'children'),
     Output('breed-chart', 'figure'),
     Output('age-chart', 'figure'),
     Output('outcome-chart', 'figure')],
    [Input('datatable-id', 'data')]
)
def update_statistics(data):
    """
    Updates statistics cards and charts based on current data.
    
    Args:
        data: Current data in the table
        
    Returns:
        Tuple containing statistics values and chart figures
    """
    df = pd.DataFrame(data)
    
    if df.empty:
        return "0", "0", "0", "N/A", "0%", {}, {}, {}
    
    # Calculate statistics
    total_animals = len(df)
    avg_age_years = df['age_upon_outcome_in_weeks'].mean() / 52.0
    unique_breeds = df['breed'].nunique()
    common_sex = df['sex_upon_outcome'].mode().iloc[0]
    
    # Calculate positive outcome rate
    positive_outcomes = ['Return to Owner', 'Adoption']
    positive_rate = (df['outcome_type'].isin(positive_outcomes).mean() * 100)
    
    # Create breed distribution chart
    breed_counts = df['breed'].value_counts().head(10)
    breed_fig = go.Figure(data=[
        go.Bar(
            x=breed_counts.values,
            y=breed_counts.index,
            orientation='h',
            marker_color=colors['breed_chart'],
            hovertemplate='<b>%{y}</b><br>' +
                         'Count: %{x} animals<extra></extra>'
        )
    ])
    breed_fig.update_layout(
        title={
            'text': 'Top 10 Breeds',
            'font': {'size': 24}
        },
        xaxis_title='Number of Animals',
        yaxis_title='Breed',
        height=400,
        font={'size': 12},
        hoverlabel={'font_size': 14},
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    breed_fig.update_xaxes(gridcolor='lightgray')
    breed_fig.update_yaxes(gridcolor='lightgray')

    # Create age distribution chart
    def categorize_age(weeks):
        """Converts age from weeks to a readable format."""
        years = weeks / 52.0
        if years < 1:
            months = int(years * 12)
            return f"{months} month{'s' if months != 1 else ''}"
        else:
            return f"{int(years)} year{'s' if int(years) != 1 else ''}"
    
    df['age_category'] = df['age_upon_outcome_in_weeks'].apply(categorize_age)
    
    def get_sort_key(age_str):
        """Creates a numerical key for sorting age categories."""
        if 'month' in age_str:
            return int(age_str.split()[0]) / 12
        else:
            return int(age_str.split()[0])
    
    age_counts = df['age_category'].value_counts()
    age_counts = age_counts.sort_index(key=lambda x: x.map(get_sort_key))
    
    age_fig = go.Figure(data=[
        go.Bar(
            x=age_counts.index,
            y=age_counts.values,
            marker_color=colors['age_chart'],
            hovertemplate='<b>%{x}</b><br>' +
                         'Count: %{y} animals<extra></extra>'
        )
    ])
    
    age_fig.update_layout(
        title={
            'text': 'Age Distribution',
            'font': {'size': 24}
        },
        xaxis_title='Age Category',
        yaxis_title='Number of Animals',
        height=400,
        font={'size': 12},
        hoverlabel={'font_size': 14},
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis={'tickangle': 45}
    )
    age_fig.update_xaxes(gridcolor='lightgray', type='category')
    age_fig.update_yaxes(gridcolor='lightgray')
    
    # Create outcome distribution chart
    outcome_counts = df['outcome_type'].value_counts()
    
    outcome_fig = go.Figure(data=[
        go.Pie(
            labels=outcome_counts.index,
            values=outcome_counts.values,
            marker={'colors': [colors['outcome_chart'].get(outcome, 'rgb(128, 128, 128)') 
                             for outcome in outcome_counts.index]},
            hovertemplate='<b>%{label}</b><br>' +
                         'Count: %{value} animals<br>' +
                         'Percentage: %{percent}<extra></extra>'
        )
    ])
    
    outcome_fig.update_layout(
        title={
            'text': 'Outcome Distribution',
            'font': {'size': 24}
        },
        height=400,
        font={'size': 12},
        hoverlabel={'font_size': 14},
        showlegend=True,
        legend={'orientation': 'h', 'yanchor': 'bottom', 'y': -0.2}
    )
    
    return (
        f"{total_animals:,}",
        f"{avg_age_years:.1f} years",
        f"{unique_breeds:,}",
        common_sex,
        f"{positive_rate:.1f}%",
        breed_fig,
        age_fig,
        outcome_fig
    )    

# Callback for Data Table Updates
@app.callback(
    [Output('datatable-id', 'data'),
     Output('page-display', 'children'),
     Output('page-data', 'data'),
     Output('page-input', 'max'),
     Output('page-input', 'value')],
    [Input('rescue-type-filter', 'value'),
     Input('age-range-slider', 'value'),
     Input('page-size-dropdown', 'value'),
     Input('prev-page-button', 'n_clicks'),
     Input('next-page-button', 'n_clicks'),
     Input('page-input', 'value'),
     Input('datatable-id', 'sort_by')],
    [State('page-data', 'data')]
)
def update_table(rescue_type, age_range, page_size, prev_clicks, next_clicks, page_input, sort_by, current_page_data):
    """
    Updates the data table based on filters, pagination, and sorting.
    
    Args:
        rescue_type: Selected rescue type filter
        age_range: Selected age range [min, max]
        page_size: Number of entries per page
        prev_clicks: Previous button click count
        next_clicks: Next button click count
        page_input: Direct page number input
        sort_by: Sorting configuration
        current_page_data: Current pagination state
    """
    ctx = dash.callback_context
    
    # Initialize current page
    if not current_page_data:
        current_page = 1
    else:
        current_page = current_page_data.get('current_page', 1)
    
    # Handle page changes based on user interaction
    if ctx.triggered:
        trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if trigger_id in ['rescue-type-filter', 'age-range-slider', 'page-size-dropdown', 'datatable-id']:
            current_page = 1
        elif trigger_id == 'prev-page-button' and current_page > 1:
            current_page -= 1
        elif trigger_id == 'next-page-button':
            current_page += 1
        elif trigger_id == 'page-input' and page_input is not None:
            current_page = page_input

    # Build query based on filters
    query = {}
    
    # Apply age range filter (convert years to weeks)
    min_age_weeks = age_range[0] * 52
    max_age_weeks = age_range[1] * 52
    query['age_upon_outcome_in_weeks'] = {
        '$gte': min_age_weeks,
        '$lte': max_age_weeks
    }
    
    # Apply rescue type filter if specified
    if rescue_type != 'All':
        criteria = rescue_criteria[rescue_type]
        query.update({
            'breed': {'$in': criteria['breeds']},
            'sex_upon_outcome': criteria['sex']
        })
    
    # Get paginated and sorted data
    result = shelter.read(query, page=current_page, page_size=page_size, sort_by=sort_by)
    
    # Convert to DataFrame for date formatting
    df = pd.DataFrame(result['data'])
    
    # Format dates
    if not df.empty:
        df['date_of_birth'] = pd.to_datetime(df['date_of_birth']).dt.strftime('%m-%d-%Y')
        df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%m-%d-%Y')
    
    # Ensure current page is valid
    total_pages = max(1, result['metadata']['total_pages'])
    current_page = min(max(1, current_page), total_pages)
    
    # Update page display
    page_display = f"of {total_pages}"
    
    # Store current page data
    page_data = {
        'current_page': current_page,
        'total_pages': total_pages
    }
    
    return df.to_dict('records'), page_display, page_data, total_pages, current_page

# Callback to Update Page Size
@app.callback(
    Output('datatable-id', 'page_size'),
    [Input('page-size-dropdown', 'value')]
)
def update_page_size(selected_size):
    """Updates the number of entries shown per page."""
    return selected_size

# Callback to Update Age Range Slider Display
@app.callback(
    Output('slider-output-container', 'children'),
    [Input('age-range-slider', 'value')]
)
def update_slider_output(value):
    """
    Formats and displays the selected age range in a readable format.
    
    Args:
        value: List containing [min_age, max_age] in years
        
    Returns:
        Formatted string describing the selected age range
    """
    def format_age(years):
        """Converts years to a more readable format."""
        if years == 0:
            return "0 years"
        elif years == 1:
            return "1 year"
        elif years % 1 == 0:
            return f"{int(years)} years"
        else:
            years_part = int(years)
            months_part = int((years % 1) * 12)
            if years_part == 0:
                return f"{months_part} months"
            else:
                year_text = "year" if years_part == 1 else "years"
                return f"{years_part} {year_text} and {months_part} months"
    
    return f"Selected age range: {format_age(value[0])} to {format_age(value[1])}"

# Server Configuration
if __name__ == '__main__':
    port = 8050
    print("\n=== Grazioso Salvare Animal Dashboard ===")
    print("\nInitializing server...")
    print(f"Dashboard ready! Access it at: http://127.0.0.1:{port}/")
    print("\nTo use the dashboard:")
    print("• Click the link above or copy it to your web browser")
    print("• The dashboard will open in a new browser window")
    print("\nTo stop the server:")
    print("• Option 1: Click the 'Stop' button (■) in the toolbar")
    print("• Option 2: Click the 'Restart' button (⟳) in the toolbar")
    print("\nStarting server...")
    app.run_server(debug=True, port=port)

Setting up the dashboard...

=== Grazioso Salvare Animal Dashboard ===

Initializing server...
Dashboard ready! Access it at: http://127.0.0.1:8050/

To use the dashboard:
• Click the link above or copy it to your web browser
• The dashboard will open in a new browser window

To stop the server:
• Option 1: Click the 'Stop' button (■) in the toolbar
• Option 2: Click the 'Restart' button (⟳) in the toolbar

Starting server...
